# Homework №11

## Imports

In [1]:
!pip install datasets tokenizers wandb xformers sentencepiece zstandard jsonlines -q

In [2]:
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_scheduler,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    pipeline
)
from datasets import load_dataset

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from IPython import display
from IPython.display import clear_output

from tqdm.auto import trange, tqdm
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import wandb

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [5]:
wandb.login()
wandb.init(
    project='actprob-hw11',
)

wandb: Currently logged in as: tokubetsu01. Use `wandb login --relogin` to force relogin


## Model

Читаем токенайзер и модел. Я решила учить rut5-small, так как когда-то работала с ним на датасете SQUAD и получилось неплохо. Плюс по опыту я помню, что на полное дообучение она чуть ли не единственная из T5 братии, кто нормально влезает в память колаба

In [6]:
tokenizer = T5Tokenizer.from_pretrained("cointegrated/rut5-small")
model = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-small")
model.to(device);

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
You are using a mod

## Dataset

In [6]:
data = load_dataset('IlyaGusev/ru_turbo_alpaca', split='train')
data = data.filter(lambda x: x['label'] != 'bad_output')
data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for IlyaGusev/ru_turbo_alpaca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/ru_turbo_alpaca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `d

Dataset({
    features: ['instruction', 'input', 'output', 'alternative_output', 'label', 'all_labels', 'agreement', 'overlap'],
    num_rows: 29132
})

In [7]:
def tokenize(item):
    res = {'input_ids': [], 'attention_mask': [], 'labels': []}
    for instr, inp, out in zip(item['instruction'], item['input'],
                                        item['output']):
        smpl = tokenizer(instr, inp,
                            padding='max_length',
                            truncation='longest_first',
                            max_length=MAX_LEN,
                            return_tensors='pt')
        target = tokenizer(out,
                            padding='max_length',
                            truncation=True,
                            max_length=MAX_LEN,
                            return_tensors='pt')

        res['input_ids'].append(smpl.input_ids)
        res['attention_mask'].append(smpl.attention_mask)
        res['labels'].append(target.input_ids)
    return res

In [9]:
MAX_LEN = 128

ds = data.map(tokenize, batched=True)

In [10]:
BATCH_SIZE = 64

dl = ds.remove_columns(['instruction', 'input', 'output', 'alternative_output', 'label', 'all_labels', 'agreement', 'overlap'])
dl.set_format("torch")
dl = dl.shuffle(seed=42)
dl = DataLoader(dl, shuffle=False, batch_size=BATCH_SIZE)

## Training

In [12]:
optimizer = AdamW(model.parameters(), lr=1e-4)

num_epochs = 3
num_training_steps = num_epochs * len(train)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [16]:
def train(model, dl, optimizer, lr_scheduler,
          model_path='/content/drive/MyDrive/nnlp/actprob/model_hw11',
          step=400, name='Loss'):
    global losses

    model.train()

    for epoch in trange(num_epochs):
        tq = tqdm(total=len(dl))
        for num, batch in enumerate(dl):
            batch = {k: v.resize(v.size(0), MAX_LEN).to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            losses.append(loss.detach().cpu())
            tq.set_postfix({'Loss': (sum(losses[-10:]) / len(losses[-10:])).item()})
            tq.update(1)
            loss.backward()

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

            if num % step == 0:
                model.save_pretrained(model_path)

            wandb.log({name: loss.detach().cpu()})

    model.eval()

In [ ]:
losses = []
train(model, dl, optimizer, lr_scheduler)

In [14]:
model_path = '/content/drive/MyDrive/nnlp/actprob/model_hw11_t5'
model.save_pretrained(model_path)

In [16]:
generator = pipeline("text2text-generation",
                     model=model,
                     tokenizer=tokenizer)

Попробуем на примерах из датасета. Получается очень плохо, хотя этого следовало ожидать от маленького T5 и трех эпох. Хотя с поиском ответов в тексте он когда-то хорошо справлялся.

In [22]:
generator('Опиши процесс изготовления торта. Укажи ингредиенты, необходимые для этого.', max_new_tokens=150)

[{'generated_text': 'Для приготовления торта нужно приготовить ингредиенты, которые нужно для приготовления торта. Для приготовления торта нужно приготовить ингредиенты, которые нужно для этого. Для приготовления торта нужно приготовить ингредиенты, которые нужно для этого. Для приготовления торта нужно приготовить ингредиенты, которые нужно для этого. Для приготовления торта нужно приготовить ингредиенты, которые нужно для приготовления торта нужно приготовить ингредиенты, которые нужно для приготовления торта. Для приготовления торта нужн'}]

In [26]:
generator('Опиши процесс получения золота из золотосодержащей руды.', max_new_tokens=150)

[{'generated_text': 'Для получения золота из золотосодержащей руды нужно получить золота из золотосодержащей руды. После получения золота нужно получить золота из золотосодержащей руды. После получения золота нужно получить золота из золотосодержащей руды. После получения золота нужно получить золота из золотосодержащей руды, чтобы получить золота из золотосодержащей руды. После получения золота нужно получить золота из золотосодержащей руды, чтобы получить золота из золотосодержащей руды. После получения золота нужно получить золота из золото'}]

In [28]:
generator('Расскажи, как готовится пицца.', max_new_tokens=150)

[{'generated_text': 'Для приготовления пицца нужно приготовить пицца, а также добавить соль и соль. Для приготовления пицца нужно приготовить соль, соль и соль. Для приготовления пицца нужно приготовить соль, соль и соль. Для приготовления пицца нужно приготовить соль, соль и соль. Для приготовления пицца нужно приготовить соль, соль и соль. Для приготовления пицца нужно приготовить соль, соль и соль. Для приготовления пицца нужно приготовить соль, соль и соль.'}]

Попробуем что-то свое.

In [29]:
generator('Расскажи, как покормить кошку.', max_new_tokens=150)

[{'generated_text': 'Для покормки кошки нужно покормить кошку, чтобы покормить кошку. Для покормки кошки нужно покормить кошку, чтобы покормить кошку. Для покормки нужно покормить кошку, чтобы покормить кошку. Для покормки нужно покормить кошку, чтобы покормить кошку и покормить кошку. Для покормки нужно покормить кошку, чтобы покормить кошку и покормить кошку. Для покормки нужно покормить кошку, чтобы покормить кошку можно покормить.'}]

In [31]:
generator('Отсортируй список по возрастанию: [12, 5, 8, 14, 2].', max_new_tokens=150)

[{'generated_text': '[12, 5, 8, 14, 2] - это множество видов спорта, которые можно использовать для того, чтобы повысить уровень роста и роста.'}]

Все очень плохо, к сожалению. Хотя мне теперь крайне интереснно, откуда модель выучила тот странный паттерн перечисления, который выдает фактически в каждом ответе.

Но вот это "Для приготовления торта нужно приготовить ингредиенты, которые нужно для приготовления торта.", я считаю, лучшая часть выдачи модели.

In [33]:
model.to('cpu');

In [34]:
import gc

del model
gc.collect()
torch.cuda.empty_cache()

## Decoder model

А потом я внимательно прочитала задание и увидела, что обучать надо было что-то из декодеров.

In [11]:
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"

tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)
model.to(device);

In [ ]:
MAX_LEN = 64

ds = data.map(tokenize, batched=True)

In [13]:
BATCH_SIZE = 32

dl = ds.remove_columns(['instruction', 'input', 'output', 'alternative_output', 'label', 'all_labels', 'agreement', 'overlap'])
dl.set_format("torch")
dl = dl.shuffle(seed=42)
dl = DataLoader(dl, shuffle=False, batch_size=BATCH_SIZE)

In [14]:
optimizer = AdamW(model.parameters(), lr=1e-4)

num_epochs = 3
num_training_steps = num_epochs * len(dl)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [17]:
losses = []
train(model, dl, optimizer, lr_scheduler, name='GPT Loss')

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/911 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:868: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:868: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:868: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:868: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


  0%|          | 0/911 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:868: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:868: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:868: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


  0%|          | 0/911 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:868: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:868: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:868: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [18]:
model_path = '/content/drive/MyDrive/nnlp/actprob/model_hw11_gpt'
model.save_pretrained(model_path)

In [20]:
generator = pipeline("text-generation",
                     model=model,
                     tokenizer=tokenizer)

In [35]:
model.to(device);

In [46]:
text = 'Привет! Как жизнь?'
toks = tokenizer(text, return_tensors='pt')
model(**{k: v.to(device) for k, v in toks.items()}).logits.squeeze().argmax(axis=1)

tensor([0, 0, 0, 0, 0], device='cuda:0')

In [26]:
generator('Привет! ', max_new_tokens=150)

[{'generated_text': 'Привет! '}]

Но с гпт что-то глобально пошло не так на этапе обучения, так как он просто ничего теперь не генерирует. Кажется, придется все-таки считать, что Т5 является основной моделью, пусть и работает очень плохо...